In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression as lg

In [2]:
#enter the path to the train dataset
path = ''

In [3]:
raw_data = pd.read_csv(path, index_col = 'competitorname')

In [4]:
raw_data

,chocolate,fruity,caramel,peanutyalmondy,nougat,crispedricewafer,hard,bar,pluribus,sugarpercent,pricepercent,winpercent,Y
competitorname,,,,,,,,,,,,,
100 Grand,1,0,1,0,0,1,0,1,0,0.732,0.860,66.971725,1
3 Musketeers,1,0,0,0,1,0,0,1,0,0.604,0.511,67.602936,1
One dime,0,0,0,0,0,0,0,0,0,0.011,0.116,32.261086,0
One quarter,0,0,0,0,0,0,0,0,0,0.011,0.511,46.116505,0
Air Heads,0,1,0,0,0,0,0,0,0,0.906,0.511,52.341465,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
Snickers Crisper,1,0,1,1,0,1,0,1,0,0.604,0.651,59.529251,1
Sour Patch Kids,0,1,0,0,0,0,0,0,1,0.069,0.116,59.863998,1
Sour Patch Tricksters,0,1,0,0,0,0,0,0,1,0.069,0.116,52.825947,1


In [5]:
#enter competitornames from the task to make data to train the model
data_train = raw_data.drop(['Kit Kat', 'Milk Duds', 'Sour Patch Tricksters'])

In [17]:
#splitting the data to build the classifier 
x_train = pd.DataFrame(data_train.drop(['winpercent', 'Y'], axis = 1))
y_train = pd.DataFrame(data_train['Y'], index = data_train.index)
y_train = np.array(y_train)
y_train = y_train.ravel()

In [16]:
clf = lg(random_state = 2019, solver = 'lbfgs').fit(x_train, y_train)

In [21]:
#enter the path to test dataset
path_test = ''
data_test = pd.read_csv(path_test, index_col = 'competitorname')

In [22]:
x_test = pd.DataFrame(data_test.drop(['Y'], axis = 1))
y_test = np.array(data_test['Y'])
y_test = y_test.ravel()

In [56]:
#returns accuracy, useless here
score = clf.score(x_test, y_test)
score

0.6666666666666666

In [27]:
#data to check probabilties
name_1 = ''
name_2 = ''
i = 0
for each in data_test.index:
    if name_1 in each:
        n_1_ind = i
        i += 1
    elif name_2 in each:
        n_2_ind = i
        i += 1
    else:
        i += 1
        
test_d = pd.DataFrame([data_test.iloc[n_1_ind],data_test.iloc[n_2_ind]], index = [name_1, name_2])


In [30]:
#probablities class [0, 1]
predict_prob = clf.predict_proba(test_d.drop(['Y'], axis = 1))
predict_prob

array([[0.332352 , 0.667648 ],
       [0.6890034, 0.3109966]])

In [35]:
#predict labels
y_pred = clf.predict(x_test)  

In [49]:
y_pred_ser = pd.Series(y_pred, name = 'y_pred', index = data_test.index)
y_test_ser = pd.Series(y_test, name = 'y_test', index = data_test.index)
conf_df = pd.DataFrame(y_test_ser)
conf_df = conf_df.merge(y_pred_ser, right_index = True, left_index = True)
conf_df

,y_test,y_pred
competitorname,,
Sugar Babies,0,0
Sugar Daddy,0,0
Super Bubble,0,0
Swedish Fish,1,0
Tootsie Pop,1,1
Tootsie Roll Juniors,0,1
Tootsie Roll Midgies,0,1
Tootsie Roll Snack Bars,1,1
Trolli Sour Bites,0,0


In [32]:
from sklearn.metrics import confusion_matrix as confuse 

In [54]:
#creating cofusion matrix and assigning true negative
#false positive, false negative and true positive results
tn, fp, fn, tp = confuse(conf_df['y_test'], conf_df['y_pred']).ravel()

In [57]:
recall = tp/(tp + fn)
precision = tp/(tp + fp)
recall, precision

(0.6, 0.5)

In [64]:
from sklearn.metrics import roc_auc_score as auc

In [69]:
#calculating AUC
res_auc = auc(y_test, clf.decision_function(x_test))
res_auc

0.8799999999999999